In [48]:
from pydantic import Field, BaseModel
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## Langsmith Tracking And Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"


In [49]:
class Product(BaseModel):
    name:str=Field(description="Provide actual name of the product")
    # details: str = Field(description="Provide high level summary of the product")
    price: int = Field(description="provide tentative price of the product in USD")
    

In [50]:
from langchain_groq import ChatGroq
model=ChatGroq(model="qwen-qwq-32b")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10b649eb0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10b7708c0>, model_name='qwen-qwq-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [51]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_messages(
    [
    ("system", "You are a sales helpful assistant. provide details for asked products in json"),
    ("human","{product}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['product'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a sales helpful assistant. provide details for asked products in json'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], input_types={}, partial_variables={}, template='{product}'), additional_kwargs={})])

In [52]:
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser,PydanticOutputParser
# output_parser=PydanticOutputParser(pydantic_object=Product)
output_parser=JsonOutputParser(pydantic_object=Product)
new_parser=JsonOutputParser(pydantic_object=Product)
from langchain_core.prompts import PromptTemplate
# prompt=PromptTemplate(
#     template="Answer the user query in json\n {format_instruction}\n {query}\n as an sales assistant ",
#     input_variables=["query"],
#     partial_variables={"format_instruction":output_parser.get_format_instructions()},
# )


In [55]:
output_parser

JsonOutputParser(pydantic_object=<class '__main__.Product'>)

In [53]:
chain=prompt|model|output_parser

In [54]:
print(chain.invoke({"product":"tell me about iPhone 15"}))


OutputParserException: Invalid json output: <think>
Okay, the user is asking about the iPhone 15. Let me recall what I know about it. I should start with the release date. I think it came out in September 2023. The main features... the display is always a key point. They probably used the same Dynamic Island but maybe a different material? Wait, I remember they switched to ceramic front glass instead of stainless steel frames. That's a new material, so that's important to mention.

The processor is the A17 Pro chip, which is a major upgrade. They emphasized performance and efficiency here. The camera system: the main camera might have had some improvements. Oh right, the 48MP sensor, which allows for more detailed photos and maybe better video. The Pro models might have a 50MP Telephoto, and the regular models could have a 12MP or 14MP? Wait, the user might be interested in the Pro variants. Also, the Night mode and portrait mode improvements should be highlighted.

Battery life is alw